## Spam message classification

As part of this assignment , we are analysing the spam messages for classifcation

**Data source**
https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

Link to the recording:

https://www.youtube.com/watch?v=tkuucWzI_Ts

In [68]:
# load the required libraries
import nltk
from nltk.tokenize import word_tokenize
import urllib, re, nltk, string
import matplotlib.pyplot as plt
% matplotlib inline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

**Load Dataset**

The Data set contains 5574 text messages which are classified as either spam or harm


In [74]:
# Read the file line by into list
#with open('D:\Personal\Masters\Rathish-Personal\CUNY\Summer-22\Rathish\Week5Assignment\SMSSpamCollection') as f:
#    lines = [line.rstrip() for line in f]
  
lines =[]
for line in urllib.request.urlopen('https://raw.githubusercontent.com/rathish-ps/Data620/main/SMSSpamCollection'):
    lines.append(line.decode('utf-8').rstrip())

## Pre Processing
Process the list if messages line by line and covert it into a list of tuples (message , label)


In [88]:
print(lines[0])
all_messages =[(re.split(r'\t+', message)[1],re.split(r'\t+', message)[0]) for message in lines]
print(all_messages[:3])

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'ham'), ('Ok lar... Joking wif u oni...', 'ham'), ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'spam')]


## Feature extraction

We are now going to seperate the spam and ham messages to identify the top words from each category 
after removing the stop words from it


In [76]:
#we need to remove the stop words
stop_words = set(stopwords.words('english'))

In [77]:

ham_messages =''
spam_messages =''
for (msg, cls) in all_messages:
    if cls =='ham':
      ham_messages =   ham_messages +msg
    if cls =='spam':
      spam_messages =   spam_messages +msg
    
#print(ham_messages)    

#identify the top 200 words from ham and spam messages
pattern = r'\w{2,}' #we want to remove unwanted characters like *, # etc
ham_tokens = nltk.regexp_tokenize(ham_messages, pattern)
ham_text = nltk.Text(ham_tokens)
ham_text_lower = [word.lower() for word in ham_text if word not in stop_words]
ham_wordFreq = nltk.FreqDist(ham_text_lower)
ham_top200 = ham_wordFreq.most_common(200)
print(ham_top200[:10])

    

[('gt', 318), ('lt', 316), ('get', 301), ('go', 249), ('ur', 247), ('ok', 235), ('know', 231), ('like', 229), ('got', 228), ('call', 227)]


In [78]:
pattern = r'\w{2,}' #we want to remove unwanted characters like *, # etc
spam_tokens = nltk.regexp_tokenize(spam_messages, pattern)
spam_text = nltk.Text(spam_tokens)
spam_text_lower = [word.lower() for word in spam_text if word not in stop_words]
spam_wordFreq = nltk.FreqDist(spam_text_lower)
spam_top200 = spam_wordFreq.most_common(200)
print(spam_top200[:10])


[('call', 344), ('free', 196), ('txt', 160), ('ur', 134), ('mobile', 126), ('text', 120), ('claim', 112), ('stop', 112), ('reply', 101), ('www', 98)]


In [79]:
#remove the common words from top 200 spam and ham words if any.
def remove_common(a, b):
  
    for i in a[:]:
        if i in b:
            a.remove(i)
            b.remove(i)
            
remove_common(ham_top200,spam_top200)            

In [80]:
# we are preparting the feature set based on the top most of words from each category
word_features = [x for x,y in ham_top200] + [x for x,y in spam_top200]


After preprocessing, we need to extract feature from text message. So it is necessary to tokenize message and verify if it is present in the featureset.

In [81]:
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

In [82]:
# Call find_features function for each SMS message
feature_set = [(find_features(text), label) for (text, label) in all_messages]

## Modelling

We are now going to split the data into training and test set and build a classifier

In [83]:

train_set, test_set = train_test_split(feature_set, test_size=0.25, random_state=1)

In [84]:
print(len(train_set))
print(len(test_set))

4180
1394


In [85]:
#DecisionTreeClassifier 
classifier = nltk.DecisionTreeClassifier.train(train_set)
nltk.classify.accuracy(classifier, test_set)

0.9677187948350072

In [67]:
print(classifier.classify((test_set[0])[0]))
print(classifier.classify((test_set[0])[0]) ==(test_set[0])[1])



ham
True


In [86]:
nbClsfr = nltk.NaiveBayesClassifier.train(train_set)
print (nltk.classify.accuracy(nbClsfr, test_set))


0.9741750358680057


In [52]:
nbClsfr.show_most_informative_features(5)

Most Informative Features
                 service = True             spam : ham    =    142.7 : 1.0
                      16 = True             spam : ham    =     87.3 : 1.0
                   award = True             spam : ham    =     57.5 : 1.0
                    draw = True             spam : ham    =     55.7 : 1.0
                selected = True             spam : ham    =     52.4 : 1.0
